# Convert Bike Count point Coordinates from GDA 94 to WGS 84
Data source uses the mapping projection system GDA 94 (A low distortion projection for the Melbourne area). 
Most web based maps, including google maps use web mercator WGS 84 (a projection used by many GPS based systems).   

This script converts the lat/lon point data in SuperTueDataCleaner/script_output/count_locations/count_location_details.csv 
to a digital web mercator for later mapping.

It also flags sites with bad coordinate data for further investigation

In [8]:
sourcefile = './script_output/count_locations/count_location_details.csv'

In [9]:
import pandas as pd

In [10]:
df = pd.read_csv(sourcefile, sep=', ', header = 0, engine = 'python')
df

,countsite,site_description,suburb,dist_from_cbd,easting,northing,melway_ref,primary_road,secondary_road
0,BW-CityLinkBrunswickRd,Brunswick Rd / Ormond Rd and Gibson St - Near ...,Brunswick West,5.7,318495.19,5817202.69,29 B6,Moonee Ponds Creek Trail,Brunswick Rd
1,BW-CityLinkDawsonSt,Dawson St and Jewel Crs - near CityLink,Brunswick West,7.1,318424.79,5817844.06,29 B8,Moonee Ponds Creek Trail,Dawson St
2,BW-CityLinkVictoriaSt,Victoria St (CityLink Overpass),Brunswick West,7.8,318346.84,5818350.13,29 E10,Moonee Ponds Creek Trail,Victoria St
3,BW-HopetounAvAlbionSt,Hopetoun Av Waxman Pde and Albion St,Brunswick West,8.8,318042.84,5819089.27,29 A4,Moonee Ponds Creek Trail,Albion St
4,BW-MelvilleDawsonSt,Melville St and Dawson St,Brunswick West,6.6,318908.20,5817775.38,29 C8,Melville Rd,Glenlyon - Dawson
5,BW-DalyStDawsonSt,Daly St South Daly St and Dawson St,Brunswick West,6.6,318907.57,5817774.70,29 C8,Daly St,Glenlyon - Dawson
6,BW-FlemingStParkSt,Fleming St and Park St,Brunswick West,5.6,319031.96,5817025.18,29 D10,Fleming St,Park St
7,BW-55TramParkSt,55 Tram and Park St,Brunswick West,5.2,319428.72,5816965.79,29 E10,West Brunswick Shimmy,Park St
8,BW-GranthamStBrunswickRd,Grantham St and Brunswick Rd,Brunswick,5.5,319352.37,5817091.00,29 D9,West Brunswick Shimmy,Brunswick Rd
9,BW-MelvilleRdVictoriaSt,Melville Rd and Victoria St,Brunswick West,7.2,318883.04,5818287.17,29 C6,Melville Rd,Victoria St


In [11]:
# GeoJSON will fail if there are no lat long coordinates,  drop these rows from the dataframe.
missingdata = df[df['easting'].isnull()]
missingdata

,countsite,site_description,suburb,dist_from_cbd,easting,northing,melway_ref,primary_road,secondary_road
34,BE-CanningBrunswick,Canning Street x Brunswick Rd Brunswick East,Brunswick East,4.9,NaN,NaN,30 A10,East Brunswick Shimmy,Brunswick Road


## Use pyproj to convert point coordinates

In [12]:
df = df.drop(missingdata.index)

In [13]:
import pyproj


def gda94_to_wgs84(series):
    '''
    Takes a series (a row of df), looks for 'easting' and 'northing' values
    Series must contain 'easting' and 'northing' values to work
    Creates an tuple 'lat lon' 
    
    '''
    easting = series['easting']
    northing = series['northing']
    
    gda94 = pyproj.Proj(init='epsg:28355')
    wgs84 = pyproj.Proj(init='epsg:4326')
    longitude, latitude = pyproj.transform(gda94, wgs84, easting, northing)

    latitude = round(latitude, 4)
    longitude = round(longitude, 4)
    series['latitude'] = latitude
    series['longitude'] = longitude
    return series


    

In [14]:
sitepoints = df.apply(gda94_to_wgs84,axis='columns')
sitepoints

,countsite,site_description,suburb,dist_from_cbd,easting,northing,melway_ref,primary_road,secondary_road,latitude,longitude
0,BW-CityLinkBrunswickRd,Brunswick Rd / Ormond Rd and Gibson St - Near ...,Brunswick West,5.7,318495.19,5817202.69,29 B6,Moonee Ponds Creek Trail,Brunswick Rd,-37.7745,144.9391
1,BW-CityLinkDawsonSt,Dawson St and Jewel Crs - near CityLink,Brunswick West,7.1,318424.79,5817844.06,29 B8,Moonee Ponds Creek Trail,Dawson St,-37.7687,144.9385
2,BW-CityLinkVictoriaSt,Victoria St (CityLink Overpass),Brunswick West,7.8,318346.84,5818350.13,29 E10,Moonee Ponds Creek Trail,Victoria St,-37.7641,144.9377
3,BW-HopetounAvAlbionSt,Hopetoun Av Waxman Pde and Albion St,Brunswick West,8.8,318042.84,5819089.27,29 A4,Moonee Ponds Creek Trail,Albion St,-37.7574,144.9344
4,BW-MelvilleDawsonSt,Melville St and Dawson St,Brunswick West,6.6,318908.20,5817775.38,29 C8,Melville Rd,Glenlyon - Dawson,-37.7694,144.9439
5,BW-DalyStDawsonSt,Daly St South Daly St and Dawson St,Brunswick West,6.6,318907.57,5817774.70,29 C8,Daly St,Glenlyon - Dawson,-37.7694,144.9439
6,BW-FlemingStParkSt,Fleming St and Park St,Brunswick West,5.6,319031.96,5817025.18,29 D10,Fleming St,Park St,-37.7762,144.9451
7,BW-55TramParkSt,55 Tram and Park St,Brunswick West,5.2,319428.72,5816965.79,29 E10,West Brunswick Shimmy,Park St,-37.7768,144.9496
8,BW-GranthamStBrunswickRd,Grantham St and Brunswick Rd,Brunswick,5.5,319352.37,5817091.00,29 D9,West Brunswick Shimmy,Brunswick Rd,-37.7757,144.9488
9,BW-MelvilleRdVictoriaSt,Melville Rd and Victoria St,Brunswick West,7.2,318883.04,5818287.17,29 C6,Melville Rd,Victoria St,-37.7648,144.9438


In [15]:
# Find and remove junk Lat Lon values.
# Any latitude further south than Wilsons Promontory 38.9333° S is junk

waytoosouth = sitepoints.query('latitude < -39')
waytoosouth 

,countsite,site_description,suburb,dist_from_cbd,easting,northing,melway_ref,primary_road,secondary_road,latitude,longitude
27,B-BarrowStAlbionSt,Barrow St and Albion St,Brunswick,6.9,321143.92,581869.75,29 J6,Barrow St,Albion St,-84.5657,129.8329
68,PV-LandellsRdKentRd,Kent Rd and Landells Rd,Pascoe Vale,12.9,318797.82,582300.48,17 C6,Landells Rd,Kent Rd,-84.5558,129.6332
69,PV-WestStBoundaryRd,West St Lake St and Pascoe St Boundary Rd),Pascoe Vale,13.4,318137.87,582363.37,17 B5,West St,Boundary Rd,-84.5535,129.5755


In [16]:
sitepoints= sitepoints.drop(waytoosouth.index)

# Add the results from the "multi site data analysis tool" notebook

## Import the year plus result table

In [40]:
totalbyyear = './script_output/allsites_summary/allmovesallriders7to9year.csv'
byyeardf = pd.read_csv(totalbyyear, sep=',', header = 0, engine = 'python')
byyeardf.rename(columns = {'Unnamed: 0':'countsite'}, inplace = True)

byyeardf

,countsite,2003,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,B-BarrowStAlbionSt,NaN,NaN,NaN,NaN,101.0,NaN,112.0,NaN,NaN,NaN,165.0
1,B-EwingStBrunswickRd,NaN,NaN,NaN,NaN,209.0,NaN,293.0,NaN,NaN,287.0,NaN
2,B-FraserStAlbionSt,NaN,NaN,NaN,NaN,NaN,68.0,NaN,84.0,NaN,NaN,114.0
3,B-GranthamDawsonSt,NaN,NaN,NaN,NaN,NaN,NaN,185.0,NaN,187.0,180.0,NaN
4,B-GrayBrunswick,NaN,NaN,NaN,NaN,44.0,NaN,62.0,NaN,NaN,NaN,NaN
5,B-SydneyRdAlbionSt,NaN,NaN,NaN,NaN,NaN,223.0,NaN,216.0,NaN,233.0,NaN
6,B-SydneyRdBlythSt,NaN,NaN,NaN,NaN,224.0,NaN,231.0,NaN,NaN,NaN,208.0
7,B-SydneyRdBrunswickRd,NaN,NaN,NaN,NaN,463.0,NaN,NaN,689.0,NaN,NaN,549.0
8,B-SydneyRdGlenlyonRd,NaN,0.0,NaN,458.0,NaN,281.0,NaN,NaN,NaN,459.0,NaN
9,B-SydneyRdParkSt,491.0,NaN,740.0,NaN,766.0,NaN,747.0,NaN,1063.0,NaN,NaN


## Import the growth rate derived data

In [34]:
growthcals = './script_output/allsites_summary/allmovesallriders7to9yeargrowth.csv'
growdf = pd.read_csv(growthcals, sep=',', header = 0, engine = 'python')
growdf.rename(columns = {'Unnamed: 0':'countsite'}, inplace = True)

#Replace the whitespace characters in the column names with underscores
growdf.rename(columns = {'Most recent count year':'Most_recent_count_year'}, inplace = True)
growdf.rename(columns = {'Most recent count value':'Most_recent_count_value'}, inplace = True)
growdf.rename(columns = {'annual increase':'Annual_increase'}, inplace = True)
growdf.rename(columns = {'growth rate':'Growth_rate'}, inplace = True)
growdf.rename(columns = {'Number of times counted':'Number_of_times_counted'}, inplace = True)

growdf


,countsite,Most_recent_count_year,Number_of_times_counted,Most_recent_count_value,Annual_increase,Growth_rate,rvalue
0,B-BarrowStAlbionSt,2017,3,165,11.0,6.67,0.985
1,B-EwingStBrunswickRd,2016,3,287,14.0,4.88,0.763
2,B-FraserStAlbionSt,2017,3,114,9.0,7.89,0.998
3,B-GranthamDawsonSt,2016,3,180,-1.0,-0.56,-0.545
4,B-GrayBrunswick,2013,2,62,NaN,NaN,NaN
5,B-SydneyRdAlbionSt,2016,3,233,3.0,1.29,0.585
6,B-SydneyRdBlythSt,2017,3,208,-3.0,-1.44,-0.805
7,B-SydneyRdBrunswickRd,2017,3,549,14.0,NaN,0.377
8,B-SydneyRdGlenlyonRd,2016,4,459,43.0,9.37,0.685
9,B-SydneyRdParkSt,2015,5,1063,40.0,3.76,0.905


## Join tables

In [42]:
byyeargrowth = pd.merge(byyeardf, growdf, on='countsite')
pointswithgrowth = pd.merge(sitepoints, byyeargrowth, on='countsite')
pointswithgrowth

,countsite,site_description,suburb,dist_from_cbd,easting,northing,melway_ref,primary_road,secondary_road,latitude,...,2014,2015,2016,2017,Most_recent_count_year,Number_of_times_counted,Most_recent_count_value,Annual_increase,Growth_rate,rvalue
0,BW-CityLinkBrunswickRd,Brunswick Rd / Ormond Rd and Gibson St - Near ...,Brunswick West,5.7,318495.19,5817202.69,29 B6,Moonee Ponds Creek Trail,Brunswick Rd,-37.7745,...,64.0,NaN,NaN,87.0,2017,3,87,6.0,6.90,0.964
1,BW-CityLinkDawsonSt,Dawson St and Jewel Crs - near CityLink,Brunswick West,7.1,318424.79,5817844.06,29 B8,Moonee Ponds Creek Trail,Dawson St,-37.7687,...,NaN,47.0,NaN,NaN,2015,3,47,7.0,14.89,0.997
2,BW-CityLinkVictoriaSt,Victoria St (CityLink Overpass),Brunswick West,7.8,318346.84,5818350.13,29 E10,Moonee Ponds Creek Trail,Victoria St,-37.7641,...,42.0,NaN,NaN,NaN,2014,1,42,NaN,NaN,NaN
3,BW-HopetounAvAlbionSt,Hopetoun Av Waxman Pde and Albion St,Brunswick West,8.8,318042.84,5819089.27,29 A4,Moonee Ponds Creek Trail,Albion St,-37.7574,...,NaN,NaN,75.0,NaN,2016,3,75,8.0,10.67,0.937
4,BW-MelvilleDawsonSt,Melville St and Dawson St,Brunswick West,6.6,318908.20,5817775.38,29 C8,Melville Rd,Glenlyon - Dawson,-37.7694,...,NaN,NaN,78.0,NaN,2016,3,78,4.0,5.13,0.945
5,BW-DalyStDawsonSt,Daly St South Daly St and Dawson St,Brunswick West,6.6,318907.57,5817774.70,29 C8,Daly St,Glenlyon - Dawson,-37.7694,...,NaN,NaN,133.0,NaN,2016,3,133,4.0,NaN,0.392
6,BW-FlemingStParkSt,Fleming St and Park St,Brunswick West,5.6,319031.96,5817025.18,29 D10,Fleming St,Park St,-37.7762,...,NaN,136.0,NaN,NaN,2015,2,136,NaN,NaN,NaN
7,BW-55TramParkSt,55 Tram and Park St,Brunswick West,5.2,319428.72,5816965.79,29 E10,West Brunswick Shimmy,Park St,-37.7768,...,300.0,NaN,NaN,287.0,2017,3,287,7.0,2.44,0.512
8,BW-GranthamStBrunswickRd,Grantham St and Brunswick Rd,Brunswick,5.5,319352.37,5817091.00,29 D9,West Brunswick Shimmy,Brunswick Rd,-37.7757,...,NaN,NaN,NaN,195.0,2017,3,195,11.0,5.64,0.999
9,BW-MelvilleRdVictoriaSt,Melville Rd and Victoria St,Brunswick West,7.2,318883.04,5818287.17,29 C6,Melville Rd,Victoria St,-37.7648,...,NaN,NaN,NaN,103.0,2017,4,103,5.0,4.85,0.839


## Convert to GeoJSON for use in mapping tools

###  Original Code developed by Geoff Boeing http://geoffboeing.com/2015/10/exporting-python-data-geojson/

In [47]:
def df_to_geojson(df, properties, lat='latitude', lon='longitude'):
    geojson = {'type':'FeatureCollection', 'features':[]}
    for _, row in df.iterrows():
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point',
                               'coordinates':[]}}
        feature['geometry']['coordinates'] = [row[lon],row[lat]]
        for prop in properties:
            feature['properties'][prop] = row[prop]
        geojson['features'].append(feature)
    return geojson

In [48]:
cols = ['countsite', 'site_description', 'site_description', 'dist_from_cbd','primary_road',\
        'secondary_road','Most_recent_count_year','Most_recent_count_value','Annual_increase', 'Growth_rate',\
        'Number_of_times_counted','rvalue',\
        '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017']
geojson = df_to_geojson(pointswithgrowth, cols)

In [49]:
import json

In [50]:
output_filename = './script_output/count_locations/count_location_details.json'
with open(output_filename, 'wb') as output_file:
    json.dump(geojson, output_file, indent=2) 

# The Pandas NaN value is not valid geojson.  
To fix manually change "NaN" to "null" (using a search and replace) in a txt editor.
Then copy this file into ./json directory 